In [1]:
# !pip install faiss-cpu
# !pip install langchain
# !pip install openai
# !pip install tiktoken

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader

In [3]:
import glob
path_to_trials = ''
trial_paths = glob.glob(f'{path_to_trials}/raw/*/*.json')

In [4]:
import json
trial_documents = []
documents_to_load = 100
for trial_path in trial_paths[:documents_to_load]:
    trial = json.load(open(trial_path))
    nct_id = trial['FullStudy']['Study']['ProtocolSection']['IdentificationModule']['NCTId']
    trial_documents.append(Document(page_content=json.dumps(trial['FullStudy']), metadata={'nct_id': nct_id}))

In [5]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(trial_documents, embeddings)

In [10]:
query = "Radiotherapy for prostate cancer"
docs = db.similarity_search(query, k=1)

In [11]:
json.loads(docs[0].page_content)['Study']['ProtocolSection']['DescriptionModule']

{'BriefSummary': 'This phase I trial studies the side effects and best way to give enzalutamide, radiation therapy, and hormone therapy in treating patients with intermediate or high-risk prostate cancer. Androgens can cause the growth of prostate cancer cells. Antihormone therapy, such as enzalutamide, may lessen the amount of androgens made by the body. Radiation therapy uses high energy x rays to kill tumor cells. Giving enzalutamide, radiation therapy, and hormone therapy may be an effective treatment for prostate cancer.',
 'DetailedDescription': 'PRIMARY OBJECTIVE:\n\n1) To assess the safety of the combination of neoadjuvant and concurrent enzalutamide with an luteinizing-hormone-releasing hormone (LHRH) agonist and radiation therapy.\n\nSECONDARY OBJECTIVES:\n\nTo determine the efficacy of the combination of enzalutamide with an LHRH agonist and radiation therapy using prostate specific antigen (PSA) kinetics.\nTo determine the efficacy of the combination of enzalutamide with an